# European Bond Option Pricing - HW Model Jamshidian

Jamshidian's model is a fast way to price European bond options in the HW model. Her we value an option on a coupon paying bond using the Hull-White model and comparing the tree and the Jamshidian model for European Bond Options

In [56]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [57]:
from financepy.finutils import *
from financepy.market.curves import *
from financepy.models.FinModelRatesHW import FinModelRatesHW, FinHWEuropeanCalcType
from financepy.products.bonds import *

## Set up Discount Curve

In [58]:
settlementDate = FinDate(1, 12, 2019)

Set up discount curve

In [59]:
rate = 0.05
dcType = FinDayCountTypes.THIRTY_360_BOND
fixedFreq = FinFrequencyTypes.SEMI_ANNUAL
discountCurve = FinDiscountCurveFlat(settlementDate, rate, fixedFreq, dcType)

## Set up the Bond Option

First create the bond

In [60]:
issueDate = FinDate(1,12,2018)
maturityDate = issueDate.addTenor("10Y")
coupon = 0.06
frequencyType = FinFrequencyTypes.SEMI_ANNUAL
accrualType = FinDayCountTypes.THIRTY_360_BOND
bond = FinBond(issueDate, maturityDate, coupon, frequencyType, accrualType)

Let's first price the bond on the libor curve

In [61]:
cp = bond.cleanPriceFromDiscountCurve(settlementDate, discountCurve)
fp = bond.fullPriceFromDiscountCurve(settlementDate, discountCurve)
print("Fixed Income Clean Price: %9.3f"% cp)
print("Fixed Income Full  Price: %9.3f"% fp)

Fixed Income Clean Price:   107.177
Fixed Income Full  Price:   110.177


It's par as the discount rate is the same as the swap rate and the swap cashflows are 30/360 so like bond flows.

In [62]:
bond.printFlows(settlementDate)

SUN 01 DEC 2019          3.00 
MON 01 JUN 2020          3.00 
TUE 01 DEC 2020          3.00 
TUE 01 JUN 2021          3.00 
WED 01 DEC 2021          3.00 
WED 01 JUN 2022          3.00 
THU 01 DEC 2022          3.00 
THU 01 JUN 2023          3.00 
FRI 01 DEC 2023          3.00 
SAT 01 JUN 2024          3.00 
SUN 01 DEC 2024          3.00 
SUN 01 JUN 2025          3.00 
MON 01 DEC 2025          3.00 
MON 01 JUN 2026          3.00 
TUE 01 DEC 2026          3.00 
TUE 01 JUN 2027          3.00 
WED 01 DEC 2027          3.00 
THU 01 JUN 2028          3.00 
FRI 01 DEC 2028        103.00 


## Create the Bond Options

Then define and create the options

In [63]:
expiryDate = settlementDate.addTenor("18m")
strikePrice = 105.0
faceAmount = 100.0

In [64]:
europeanCallBondOption = FinBondOption(bond, expiryDate, strikePrice,
                                       faceAmount, FinOptionTypes.EUROPEAN_CALL)

In [65]:
europeanPutBondOption = FinBondOption(bond, expiryDate, strikePrice,
                                      faceAmount, FinOptionTypes.EUROPEAN_PUT)

Let's look at the forward price to calculate intrinsic.

In [66]:
cp = bond.cleanPriceFromDiscountCurve(expiryDate, discountCurve)
fp = bond.fullPriceFromDiscountCurve(expiryDate, discountCurve)
print("Fixed Income Clean Price: %9.3f"% cp)
print("Fixed Income Full  Price: %9.3f"% fp)

Fixed Income Clean Price:   106.191
Fixed Income Full  Price:   109.191


In [67]:
df = discountCurve.df(expiryDate)

In [68]:
CallPx = max(cp - strikePrice,0) * df

In [69]:
PutPx = max(strikePrice - cp,0) * df

In [70]:
print(CallPx, PutPx)

1.1056729769751705 0.0


## Setting up Models

Set up all three bond option model

In [71]:
sigma = 0.01
a = 0.1
numSteps = 200
modelJamshidian = FinModelRatesHW(sigma, a, numSteps, FinHWEuropeanCalcType.JAMSHIDIAN)
modelExpiryOnly = FinModelRatesHW(sigma, a, numSteps, FinHWEuropeanCalcType.EXPIRY_ONLY)
modelExpiryTree = FinModelRatesHW(sigma, a, numSteps, FinHWEuropeanCalcType.EXPIRY_TREE)

## Comparing Jamshidian and Tree Implementations

In [72]:
ec1 = europeanCallBondOption.value(settlementDate, discountCurve, modelJamshidian)
ec2 = europeanCallBondOption.value(settlementDate, discountCurve, modelExpiryOnly)
ec3 = europeanCallBondOption.value(settlementDate, discountCurve, modelExpiryTree)
print("European Call Jamshidian  Value: %9.5f" % ec1)
print("European Call Expiry Only Value: %9.5f" % ec2)
print("European Call Expiry Tree Value: %9.5f" % ec3)

European Call Jamshidian  Value:   2.59431
European Call Expiry Only Value:   2.60439
European Call Expiry Tree Value:   2.60555


In [73]:
ep1 = europeanPutBondOption.value(settlementDate, discountCurve, modelJamshidian)
ep2 = europeanPutBondOption.value(settlementDate, discountCurve, modelExpiryOnly)
ep3 = europeanPutBondOption.value(settlementDate, discountCurve, modelExpiryTree)
print("European Put Jamshidian Value:  %9.5f" % ep1)
print("European Put Expiry Only Value: %9.5f" % ep2)
print("European Put Expiry Tree Value: %9.5f" % ep3)

European Put Jamshidian Value:    1.48866
European Put Expiry Only Value:   1.49790
European Put Expiry Tree Value:   1.49701


## Timing Comparisons

In [74]:
%timeit ep1 = europeanPutBondOption.value(settlementDate, discountCurve, modelJamshidian)

964 µs ± 3.39 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [75]:
%timeit ep2 = europeanPutBondOption.value(settlementDate, discountCurve, modelExpiryOnly)

2.86 ms ± 57.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [76]:
%timeit ep3 = europeanPutBondOption.value(settlementDate, discountCurve, modelExpiryTree)

628 µs ± 13.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Interestingly, the expiry tree is fastest. Possibly because the Jamshidian main loop has not been Numba'd

## Strike Dependency

In [77]:
strikes = np.linspace(70,130,31)

In [78]:
for K in strikes:
    europeanCallBondOption = FinBondOption(bond, expiryDate, K, faceAmount, FinOptionTypes.EUROPEAN_CALL)
    ec1 = europeanCallBondOption.value(settlementDate, discountCurve, modelJamshidian)
    ec2 = europeanCallBondOption.value(settlementDate, discountCurve, modelExpiryOnly)
    ec3 = europeanCallBondOption.value(settlementDate, discountCurve, modelExpiryTree)    
    print("K: %9.5f JAMSHIDIAN: %9.5f  EXPIRY_ONLY: %9.5f  EXPIRY_TREE: %9.5f " %(K, ec1, ec2, ec3))

K:  70.00000 JAMSHIDIAN:  33.60535  EXPIRY_ONLY:  33.63139  EXPIRY_TREE:  33.63344 
K:  72.00000 JAMSHIDIAN:  31.74821  EXPIRY_ONLY:  31.77282  EXPIRY_TREE:  31.77487 
K:  74.00000 JAMSHIDIAN:  29.89106  EXPIRY_ONLY:  29.91426  EXPIRY_TREE:  29.91631 
K:  76.00000 JAMSHIDIAN:  28.03392  EXPIRY_ONLY:  28.05569  EXPIRY_TREE:  28.05774 
K:  78.00000 JAMSHIDIAN:  26.17679  EXPIRY_ONLY:  26.19713  EXPIRY_TREE:  26.19917 
K:  80.00000 JAMSHIDIAN:  24.31965  EXPIRY_ONLY:  24.33856  EXPIRY_TREE:  24.34061 
K:  82.00000 JAMSHIDIAN:  22.46252  EXPIRY_ONLY:  22.48000  EXPIRY_TREE:  22.48204 
K:  84.00000 JAMSHIDIAN:  20.60539  EXPIRY_ONLY:  20.62143  EXPIRY_TREE:  20.62348 
K:  86.00000 JAMSHIDIAN:  18.74827  EXPIRY_ONLY:  18.76288  EXPIRY_TREE:  18.76493 
K:  88.00000 JAMSHIDIAN:  16.89124  EXPIRY_ONLY:  16.90441  EXPIRY_TREE:  16.90646 
K:  90.00000 JAMSHIDIAN:  15.03466  EXPIRY_ONLY:  15.04631  EXPIRY_TREE:  15.04836 
K:  92.00000 JAMSHIDIAN:  13.18003  EXPIRY_ONLY:  13.19045  EXPIRY_TREE:  13

All three methods are consistent.

Copyright (c) Dominic O'Kane 2020